In [1]:
#Installing libraries
!pip3 install split-folders
!pip3 install keras-efficientnet
!pip3 install -U efficientnet
#!pip3 install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.9 MB/s eta 0:00:00


In [2]:
# Importing libraries
import pandas as pd
import numpy as np

import os
import shutil

import cv2
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

colab = False
kaggle = True

In [3]:
# import shutil

# shutil.rmtree('/kaggle/working/dataset_final')

# shutil.rmtree('/kaggle/working/dataset')

In [4]:
if colab :
  from google.colab import drive

  # Mount Google Drive
  drive.mount('/content/drive')


In [5]:
if colab:
    train_csv = pd.read_csv("/content/drive/MyDrive/hackathon/train/train.csv")
    test_csv =  pd.read_csv("/content/drive/MyDrive/hackathon/test/test.csv")
elif kaggle:
    train_csv = pd.read_csv("/kaggle/input/dataset/train/train.csv")
    test_csv =  pd.read_csv("/kaggle/input/dataset/test/test.csv")
else:
    train_csv = pd.read_csv("./train/train.csv")
    test_csv =  pd.read_csv("./test/test.csv")

labels = {1:'crack',2:'scratch',3:'tire flat',4 :'dent', 5: 'glass shatter', 6: 'lamp broken'}

In [6]:
# Checking number of records
print(f"{train_csv.shape=}")
print(f"{test_csv.shape=}")

train_csv.shape=(7200, 3)
test_csv.shape=(4800, 2)


In [7]:
# Lets have a look at csv file
train_csv.head()

,image_id,filename,label
0,1,1.jpg,2
1,2,2.jpg,4
2,3,3.jpg,2
3,4,4.jpg,3
4,5,5.jpg,5


In [8]:
train_csv.dtypes

image_id     int64
filename    object
label        int64
dtype: object

In [9]:
# Let us see number of data points for each labels
train_csv['label'].value_counts()


label
2    2349
4    2079
5    1185
6     882
3     534
1     171
Name: count, dtype: int64

In [10]:
# Define image folder path
if colab:
    path_to_train_images = "/content/drive/MyDrive/hackathon/train/images/"
    path_to_test_images = "/content/drive/MyDrive/hackathon/test/images/"
elif kaggle:
    path_to_train_images = "/kaggle/input/dataset/train/images/"
    path_to_test_images = "/kaggle/input/dataset/test/images/"
else:
    path_to_train_images = "./train/images/"
    path_to_test_images = "./test/images/"

# Listing all images
train_image_files = os.listdir(path_to_train_images)
#test_image_files = os.listdir(path_to_test_images)

train_image_files = [t for t in train_image_files if t.endswith(".jpg")]
train_image_files=[t for t in train_image_files if '(' not in t]

In [11]:
# print(f"{train_image_files=}")
# print(f"{test_image_files=}")
#len(set(train_image_files))

In [12]:

# # Check some images of train

# for i,image_file in enumerate(train_image_files):
#     if image_file.endswith(".jpg") or image_file.endswith(".png"):
#         if i < 5:
#             print('Labels : ' , f"{labels[int(train_csv.loc[train_csv['filename'] == image_file]['label'])]}")

#             image = Image.open(path_to_train_images + image_file)
#             display(image)

#             print("-"*50)

#         else:
#             break

In [13]:
# # Let us find the maximum height and maximum weight in all images
# train_max_width = -float('inf')
# train_max_height = -float('inf')

# train_min_width = float('inf')
# train_min_height = float('inf')


# for image_file in train_image_files:
#     if image_file.endswith(".jpg") or image_file.endswith(".png"):
#         image = Image.open(path_to_train_images + image_file)

#         width, height = image.size

#         train_max_height = max(height,train_max_height)
#         train_max_width = max(width,train_max_width)

#         train_min_width = min(height,train_max_height)
#         train_min_height = min(height,train_max_height)

# print(f"{train_max_height=}")

# print(f"{train_max_width=}")

# print(f"{train_min_height=}")

# print(f"{train_min_width=}")

In [14]:
labels = {1:'crack',2:'scratch',3:'tire flat',4 :'dent', 5: 'glass shatter', 6: 'lamp broken'}
folder = './dataset/train'
train_image_files=[t for t in train_image_files if '(' not in t]

#making folder for each classes
for values in labels:
    path = os.path.join(folder,str(values))
    os.makedirs(path,exist_ok=True)

# coping our raw data to each classes

for image in train_image_files:
 #   print(image)
    label = int((train_csv[train_csv['filename'] == image]['label']).iloc[0])
  #  print(label)
  #  path = os.path.join(folder,label,'/')
    from_path = os.path.join(path_to_train_images,image)
    to_path = os.path.join(folder,str(label))
    if not os.path.exists(os.path.join(to_path,image)):
      shutil.copy(from_path,to_path)


In [15]:

import splitfolders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("./dataset/train", output="./dataset_final",
    seed=1337, ratio=(.8, .2, ), group_prefix=None, move=True) # default values


Copying files: 7200 files [00:00, 15672.51 files/s]


In [16]:
from tensorflow.keras.applications import EfficientNetB7

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

import keras
from keras import layers


IMG_SIZE = 600
batch = 32

In [18]:
# Add our data-augmentation parameters to ImageDataGenerator

train_dir = './dataset_final/train'
validation_dir = './dataset_final/val'


train_datagen = ImageDataGenerator()#rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

valid_datagen = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(train_dir, batch_size = batch, class_mode = 'categorical', target_size = (IMG_SIZE , IMG_SIZE ))

validation_generator = valid_datagen.flow_from_directory( validation_dir, batch_size = batch, class_mode = 'categorical', target_size = (IMG_SIZE, IMG_SIZE))



Found 5758 images belonging to 6 classes.
Found 1442 images belonging to 6 classes.


In [19]:
# import keras
# from keras import layers


# def build_model(num_classes):
#     inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
#     model = EfficientNetB7(include_top=False, input_tensor=inputs, weights="imagenet")

#     # Freeze the pretrained weights
#     model.trainable = False

#     # Rebuild top
#     x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
#     x = layers.BatchNormalization()(x)

#     top_dropout_rate = 0.2
#     x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
#     outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

#     # Compile
#     model = keras.Model(inputs, outputs, name="EfficientNet")
#     optimizer = keras.optimizers.Adam(learning_rate=1e-1)
#     model.compile(
#         optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
#     )
#     return model

In [20]:
# NUM_CLASSES = 6
# model = build_model(num_classes=NUM_CLASSES)



In [21]:
# #checkpoint_filepath = './ckpt/checkpoint.weights.h5'
# # include the epoch in the file name. (use `str.format`)
# checkpoint_path = './training_2/cp-{epoch:04d}.ckpt'
# checkpoint_dir = os.path.dirname(checkpoint_path)

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_path,
#     save_weights_only=True,
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=True)

In [22]:
# model.save('/content/models')

In [23]:
# epochs = 10
# hist = model.fit(train_generator, steps_per_epoch = 40,
#                  epochs=epochs, validation_data=validation_generator, callbacks=[model_checkpoint_callback])

In [24]:
model = tf.keras.models.load_model('/kaggle/input/model-keras2/my_model2.keras')

In [25]:
def unfreeze_model(model):
    # We unfreeze the top 5 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


unfreeze_model(model)



In [26]:
epochs = 30
#hist = model.fit(train_generator, epochs=epochs, validation_data=validation_generator,callbacks=[model_checkpoint_callback])
hist = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)#,callbacks=[model_checkpoint_callback])


Epoch 1/30


2023-12-02 08:45:12.434258: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inEfficientNet/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


180/180 [==============================] - 436s 2s/step - loss: 3.3507 - accuracy: 0.8765 - val_loss: 2.8266 - val_accuracy: 0.8849
Epoch 2/30
180/180 [==============================] - 397s 2s/step - loss: 3.1237 - accuracy: 0.8755 - val_loss: 2.7107 - val_accuracy: 0.8883
Epoch 3/30
180/180 [==============================] - 397s 2s/step - loss: 3.1003 - accuracy: 0.8857 - val_loss: 2.5476 - val_accuracy: 0.8925
Epoch 4/30
180/180 [==============================] - 397s 2s/step - loss: 2.9816 - accuracy: 0.8857 - val_loss: 2.6170 - val_accuracy: 0.8911
Epoch 5/30
180/180 [==============================] - 397s 2s/step - loss: 3.0203 - accuracy: 0.8784 - val_loss: 2.4577 - val_accuracy: 0.8939
Epoch 6/30
180/180 [==============================] - 397s 2s/step - loss: 2.9504 - accuracy: 0.8835 - val_loss: 2.4306 - val_accuracy: 0.8988
Epoch 7/30
180/180 [==============================] - 397s 2s/step - loss: 2.6906 - accuracy: 0.8915 - val_loss: 2.2774 - val_accuracy: 0.9015
Epoch 8/30

In [27]:
model.save('my_model4.keras')

In [28]:

# def plot_hist(hist):
#     plt.plot(hist.history["accuracy"])
#     plt.plot(hist.history["val_accuracy"])
#     plt.title("model accuracy")
#     plt.ylabel("accuracy")
#     plt.xlabel("epoch")
#     plt.legend(["train", "validation"], loc="upper left")
#     plt.show()
# plot_hist(hist)

In [29]:
# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
#model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

# The model weights (that are considered the best) can be loaded as -
#model.load_weights(checkpoint_filepath)

In [30]:
if colab:
    test_dir = '/content/drive/MyDrive/hackathon/test'
elif kaggle:
    test_dir = "/kaggle/input/dataset/test/"
else:
    test_dir = './test/'
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory( test_dir, batch_size = 20, class_mode = None, target_size = (IMG_SIZE, IMG_SIZE),shuffle= False)

test_generator.reset()


Found 4800 images belonging to 1 classes.


In [31]:
predictions = model.predict(test_generator)

predicted_class_indices=np.argmax(predictions,axis=1)
labels_map = (train_generator.class_indices)
labels_map = dict((v,k) for k,v in labels_map.items())
predictions = [labels_map[i] for i in predicted_class_indices]

240/240 [==============================] - 241s 971ms/step


In [32]:
# Preparing submitssion file

filenames=test_generator.filenames
filenames = [f[7:-4] for f in filenames]
results=pd.DataFrame({"image_id":filenames,
                      "label":predictions})
results.to_csv("sub.csv",index=False)

In [33]:
#len(filenames)

In [34]:
#set(predictions)

In [35]:
# from sklearn.metrics import f1_score
# import numpy as np
# import keras

# class DataCallback(keras.callbacks.Callback):  # diff
#     """Callback to operate on batch data from metric."""

#     def __init__(self):
#         """Offer a metric to access batch data."""
#         super().__init__()

#         self.y_true = None
#         self.y_pred = None
#         #self.metric = tf.keras.metrics.F1Score(threshold=0.5)


#     def set_model(self, model):
#         """Initialize variables when model is set."""
#         self.y_true = tf_nan(model.output.dtype)
#         self.y_pred = tf_nan(model.output.dtype)

#     def metric(self, y_true, y_pred):

#         print(y_true,y_pred)
#         """Fake metric."""
#         if self.y_true is None:
#             self.y_true.assign(y_true)
#             self.y_pred.assign(y_pred)
#         else:
#             self.y_true = tf.concat([self.y_true,y_true],axis=0)
#             self.y_pred = tf.concat([self.y_pred,y_pred],axis=0)

# #         def create_one_hot_array(y_pred):
# #         # Get the indices of the maximum values along each row
# #           max_indices = np.argmax(y_pred, axis=1)
          
# #           # Create a new array with zeros
# #           one_hot_array = np.zeros_like(y_pred)
          
# #           # Set 1 at the maximum value indices
# #           one_hot_array[np.arange(len(y_pred)), max_indices] = 1
        
# #           return one_hot_array

# #              #   _logs ['f1_Score'] = metric
# #         # print("y_true =", y_true.numpy().shape)
# #         # print("y_pred =", y_pred.numpy().shape)
        
# #         score = f1_score(y_true.numpy(),create_one_hot_array(y_pred.numpy()),average='macro')
# #         #metric.update_state(y_true.nump
# #        return score
#          return 1

#     def on_epoch_end(self, _batch, logs=None):
#         def create_one_hot_array(y_pred):
#         # Get the indices of the maximum values along each row
#             max_indices = np.argmax(y_pred, axis=1)
#             one_hot_array = np.zeros_like(y_pred)
#             one_hot_array[np.arange(len(y_pred)), max_indices] = 1
#             return one_hot_array

#         score = f1_score(self.y_true.numpy(),create_one_hot_array(self.y_pred.numpy()),average='macro')

#         print('f1_score')
#         del self.y_true, self.y_pred



#     # def on_epoch_end(self, _logs=None):
#     #     """Clean up."""
#     #     del self.y_true, self.y_pred

# def tf_nan(dtype):
#   """Create NaN variable of proper dtype and variable shape for assign()."""
#   return tf.Variable(float("nan"), dtype=dtype, shape=tf.TensorShape(None))
# f1_callback = DataCallback()

## SUBMITING OUR PREDICTIONS